# Producer

The data was generated based on the following formula:

$$
y(t+1) = -0.4 \frac{(3 - y(t)^2)}{(1 + y(t)^2)} + 0.6 \frac{(3 - (y(t-1) - 0.5)^3)}{(1 + (y(t-1) - 0.5)^3)}
$$


In [1]:
from kafka import KafkaProducer
import time
import numpy as np
import json
from message import Message
from messageManager import MessageManager
from messageManagerFactory import MessageManagerFactory

In [2]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')

In [3]:
def get_y_future(y_current, y_before):
    """
    Calculates the y(t+1) based on y(t) and y(t-1)
    
    Parameters
    ----------
    arg1: float
        represents y(t)
    arg2: float
        represents y(t-1)
        
    Returns
    -------
    float
        y(t+1)
    """
    return -0.4 * (3 - pow(y_current, 2))/(1 + pow(y_current, 2)) + 0.6 * (3 - pow(y_before - 0.5, 3)) / (1 + pow(y_before - 0.5, 3))


In [4]:
def send_message(message, producer, topic_name):
    """
    It uses the producer to sent a message using that topic's name
    
    Parameters
    ----------
    arg1: Message
        an object containing the id and the inputs xs
    arg2: KafkaProducer
    arg3: string
    
    Returns
    -------
    string
        JSON message that was sent
    """

    message_json = json.dumps(message.__dict__)
    producer.send(topic_name, message_json.encode())
    return message_json

In [6]:
np.random.seed(2452020515) # Fix seed to ensure repeatability

time_delay = 5
messages_sent = 0

message_manager_factory = MessageManagerFactory()
message_manager_factory.create(1000, topic_name = 'timeSerieS')

# y(t-1), y(t), y(t+1)
y_before = np.random.random()
y_current = np.random.random()
y_future = None

message_manager_factory.add_input(y_before)
message_manager_factory.add_input(y_current)

#Infinite loop for sending messages to Kafka with the topic_name
while True:
    
    for message_manager in message_manager_factory.get_all():
        # if the data is enough send the message
        if message_manager.has_reached_input_size_limit():
            message_manager.increment_message_sent()
            sent_message = send_message(message_manager.message, producer, message_manager.topic_name)
            print('Message on topic', message_manager.topic_name)
            print('\t ID ',message_manager.message.id,' SIZE ', message_manager.input_vector_size, ' INPUT ', message_manager.message.x[:5])
            print('\r\n')
            message_manager.reset_message()
            time.sleep(time_delay) # wait a bit to sent the other message

    y_future = get_y_future(y_current, y_before)
    y_before = y_current
    y_current = y_future

    message_manager_factory.add_input(y_future)
    

Message on topic timeSerieS
	 ID  1  SIZE  1000  INPUT  [0.0024151530439742164, 0.027939322560110558, 0.938466063764634, 1.631408777013852, 1.5764373213527065]


Message on topic timeSerieS
	 ID  2  SIZE  1000  INPUT  [-1.6230472101707256, -1.1617422500454804, -1.1610243408305332, -1.5502007771792883, -1.340029351705976]


Message on topic timeSerieS
	 ID  3  SIZE  1000  INPUT  [-1.5262923317286454, -1.374956053981285, -1.081420115911641, -1.3667432160077917, -1.5700771148941064]


Message on topic timeSerieS
	 ID  4  SIZE  1000  INPUT  [-1.3422208023888424, -1.5911690436648211, -1.109985519956949, -1.2114957929381454, -1.604713067851477]


Message on topic timeSerieS
	 ID  5  SIZE  1000  INPUT  [-1.1932808418968985, -1.589162385988133, -1.2764165896047135, -1.1041673483058292, -1.4420719304846918]


Message on topic timeSerieS
	 ID  6  SIZE  1000  INPUT  [-1.0945354646802588, -1.4156379090851727, -1.5184271775645795, -1.0820500544133709, -1.2693085902250325]


Message on topic timeSer

KeyboardInterrupt: 